In [1]:
import cv2
import numpy as np
import math
import os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector

In [2]:
# Initialize global variables
drawing = False
ix, iy = -1, -1
mask = None  # Initialize mask
save_rectangle = False

zoom_factor = 1.0
pan_x, pan_y = 0, 0
cursor_x, cursor_y = 0, 0
mouseRGB_last_x = 0
mouseRGB_last_y = 0

# Load an image
img_full = cv2.imread(r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-003_IHCB.tif')
height, width = img_full.shape[:2]
downscale_factor = 0.25
print(height, width)
img = cv2.resize(img_full, (int(width * downscale_factor), int(height * downscale_factor)), interpolation=cv2.INTER_LINEAR)
cv2.imshow('image', img)
display_img = None
coordinates = []
def mouseRGB(event, x, y, flags, param):
    global zoom_factor, pan_x, pan_y, cursor_x, cursor_y, drawing, save_rectangle, ix, iy, mask, mouseRGB_last_x, mouseRGB_last_y, display_img

    if event == cv2.EVENT_LBUTTONDOWN:  # Check mouse left button down condition
        mouseRGB_last_x, mouseRGB_last_y = x, y

    elif event == cv2.EVENT_RBUTTONDOWN:  # Check mouse right button down condition
        drawing = True
        save_rectangle = False
        ix, iy = x, y
        mask = np.zeros_like(img)  # Create a mask of zeros with the same shape as img

    elif event == cv2.EVENT_MOUSEWHEEL:
        cursor_x, cursor_y = x, y
        if flags > 0:
            zoom_factor += 0.1
        else:
            zoom_factor -= 0.1
        zoom_factor = max(0.1, zoom_factor)
        update_display_img()

    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            pan_x += x - mouseRGB_last_x
            pan_y += y - mouseRGB_last_y
            mouseRGB_last_x, mouseRGB_last_y = x, y
            update_display_img()
        elif flags & cv2.EVENT_FLAG_RBUTTON:
            if drawing:
                # Only update pan_x and pan_y if drawing is True
                mask = np.zeros_like(img)  # Reset mask to zeros
                mid_x = (ix + x) // 2
                mid_y = (iy + y) // 2
                width = abs(ix - x)
                height = abs(iy - y)
                edge_len = min(width, height)
                top_left_x = int(mid_x - edge_len // 2)
                top_left_y = int(mid_y - edge_len // 2)
                bottom_right_x = int(mid_x + edge_len // 2)
                bottom_right_y = int(mid_y + edge_len // 2)
                cv2.rectangle(mask, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 0, 255), 2)  # Draw rectangle on mask
                cv2.rectangle(mask, (ix, iy), (x, y), (0, 255, 0), 2)  # Draw rectangle on mask
                result = cv2.addWeighted(display_img, 1, mask, 0.5, 0)  # Blend original image with mask
                cv2.imshow('image', result)

    elif event == cv2.EVENT_RBUTTONUP:
        if drawing:


            drawing = False
            mask2 = np.zeros_like(img)  # Reset mask to zeros
            cv2.rectangle(mask2, (ix, iy), (x, y), (255, 0, 0), 2)  # Draw rectangle on mask
            #result = cv2.addWeighted(img, 1, mask2, 0.5, 0)  # Blend original image with mask
            mid_x = (ix + x) // 2
            mid_y = (iy + y) // 2
            width = abs(ix - x)
            height = abs(iy - y)
            edge_len = min(width, height)
            top_left_x = int(mid_x - edge_len // 2)
            top_left_y = int(mid_y - edge_len // 2)
            bottom_right_x = int(mid_x + edge_len // 2)
            bottom_right_y = int(mid_y + edge_len // 2)
            if save_rectangle:
                M = np.float32([[zoom_factor, 0, cursor_x - cursor_x * zoom_factor + pan_x],
                    [0, zoom_factor, cursor_y - cursor_y * zoom_factor + pan_y]])
                # Convert display coordinates to original image coordinates
                inv_M = cv2.invertAffineTransform(M)
                orig_top_left = cv2.transform(np.array([[[top_left_x, top_left_y]]]), inv_M)[0][0]
                orig_bottom_right = cv2.transform(np.array([[[bottom_right_x, bottom_right_y]]]), inv_M)[0][0]

                # Draw rectangle on original image
                cv2.rectangle(img,
                              (int(orig_top_left[0]), int(orig_top_left[1])),
                              (int(orig_bottom_right[0]), int(orig_bottom_right[1])),
                              (0, 0, 255), 2)
                coordinates.append([int(orig_top_left[0]), int(orig_top_left[1]), int(orig_bottom_right[0]), int(orig_bottom_right[1])]) #top x, top y, bot x, bot y
                update_display_img()


def update_display_img():
    global display_img, zoom_factor, pan_x, pan_y, cursor_x, cursor_y
    h, w = img.shape[:2]
    # Center of the zoom
    center_x, center_y = cursor_x, cursor_y
    M = np.float32([[zoom_factor, 0, center_x - center_x * zoom_factor + pan_x],
                    [0, zoom_factor, center_y - center_y * zoom_factor + pan_y]])
    display_img = cv2.warpAffine(img, M, (w, h))
    cv2.imshow('image', display_img)



cv2.setMouseCallback('image', mouseRGB)

# Wait for ESC key to exit
while True:
    key = cv2.waitKey(20) & 0xFF
    if key == 27:  # ESC key
        break
    elif key == ord('x'):
        save_rectangle = True  # Change color to blue

cv2.destroyAllWindows()

17233 21049


In [ ]:
stain_colors = ["brown", "blue", "red"]
hsv_values = [[] for _ in stain_colors]  # List to store HSV values for each color
hsv_index = 0

def set_hsv_index(value):
    global hsv_index
    try:
        hsv_index = int(value)
        if 0 <= hsv_index < len(stain_colors):
            print(f"HSV index set to: {hsv_index} ({stain_colors[hsv_index]})")
        else:
            print(f"Invalid index. Please enter a number between 0 and {len(stain_colors) - 1}.")
    except ValueError:
        print("Invalid input. Please enter a valid number.")

def mouseHSV(event, x, y, flags, param):
    global hsv_values, hsv_index

    if event == cv2.EVENT_LBUTTONDOWN:  # Check mouse left button down condition
        colorsB = display_img[y, x, 0]
        colorsG = display_img[y, x, 1]
        colorsR = display_img[y, x, 2]
        hsv_value = np.uint8([[[colorsB, colorsG, colorsR]]])
        hsv = cv2.cvtColor(hsv_value, cv2.COLOR_BGR2HSV)
        hsv = hsv[0][0]
        print("Left Click - HSV :", hsv)
        print("Coordinates of pixel: X: ", x, "Y: ", y)

    elif event == cv2.EVENT_RBUTTONDOWN:  # Check mouse right button down condition
        colorsB = display_img[y, x, 0]
        colorsG = display_img[y, x, 1]
        colorsR = display_img[y, x, 2]
        hsv_value = np.uint8([[[colorsB, colorsG, colorsR]]])
        hsv = cv2.cvtColor(hsv_value, cv2.COLOR_BGR2HSV)
        hsv = hsv[0][0]
        print("Right Click - HSV :", hsv)
        print("Coordinates of pixel: X: ", x, "Y: ", y)
        # Append HSV value to the list for the specified index
        hsv_values[hsv_index].append(hsv.tolist())
def concatenate_images(images, rows, cols):
    """
    Concatenate multiple images of the same size into a grid.
    :param images: List of images to concatenate
    :param rows: Number of rows in the grid
    :param cols: Number of columns in the grid
    :return: Concatenated image
    """
    assert len(images) <= rows * cols, "Number of images is greater than grid size"

    # Ensure all images are the same size
    assert all(img.shape == images[0].shape for img in images), "All images must have the same dimensions"

    # Create rows of images
    rows_of_images = [cv2.hconcat(images[i:i+cols]) for i in range(0, len(images), cols)]

    # Concatenate rows vertically
    return cv2.vconcat(rows_of_images)


print(coordinates)
num_imgs = len(coordinates)
x_grid = int(math.sqrt(num_imgs))
if num_imgs % x_grid == 0:
    y_grid = x_grid
else:
    y_grid = x_grid + 1
images = []
# Ask the user to input the HSV index
input_value = input("Enter a number for HSV index (0 for brown, 1 for blue, 2 for red): ")
set_hsv_index(input_value)



for coord in coordinates:
    print(f"Cropping coordinates: {coord}")
    cropped_image = img_full[int(coord[1]/downscale_factor):int(coord[3]/downscale_factor), int(coord[0]/downscale_factor):int(coord[2]/downscale_factor)]
    if cropped_image.size == 0:
        print(f"Warning: Cropped image is empty for coordinates: {coord}")
        continue
    print(cropped_image.shape)
    downsized_cropped_image = cv2.resize(cropped_image, (512, 512), interpolation=cv2.INTER_LINEAR)
    images.append(downsized_cropped_image)

if images:
    result = concatenate_images(images, y_grid, x_grid)
    cv2.namedWindow('mouseHSV', cv2.WINDOW_NORMAL)
    cv2.setMouseCallback('mouseHSV', mouseHSV)

    while True:
        cv2.imshow('mouseHSV', result)
        key = cv2.waitKey(20) & 0xFF
        if key == 27:  # ESC key to break
            break
        elif key == ord('c'):  # Press 'c' key to change HSV index
            # Ask the user to input the HSV index
            input_value = input("Enter a number for HSV index (0 for brown, 1 for blue, 2 for red): ")
            set_hsv_index(input_value)
else:
    print("No images to concatenate.")

# Initialize lists to store max and min values for each color category
max_values_list = []
min_values_list = []
# Calculate max and min values separately for each color category
for i in hsv_values:  # Iterating over each color category
    max_values = np.amax(i, axis=0).tolist()  # Convert to list for better readability
    min_values = np.amin(i, axis=0).tolist()  # Convert to list for better readability
    max_values_list.append(max_values)
    min_values_list.append(min_values)

print("Max values for each color category:", max_values_list)
print("Min values for each color category:", min_values_list)

[[3171, 574, 3271, 674], [2827, 728, 2879, 780], [3024, 929, 3088, 993], [2273, 781, 2319, 827]]
HSV index set to: 0 (brown)
Cropping coordinates: [3171, 574, 3271, 674]
(400, 400, 3)
Cropping coordinates: [2827, 728, 2879, 780]
(208, 208, 3)
Cropping coordinates: [3024, 929, 3088, 993]
(256, 256, 3)
Cropping coordinates: [2273, 781, 2319, 827]
(184, 184, 3)
Left Click - HSV : [  0   0 249]
Coordinates of pixel: X:  288 Y:  276
Left Click - HSV : [ 30   1 248]
Coordinates of pixel: X:  287 Y:  276
Left Click - HSV : [ 30   1 248]
Coordinates of pixel: X:  287 Y:  276
Left Click - HSV : [ 30   1 248]
Coordinates of pixel: X:  287 Y:  276
Left Click - HSV : [ 30   1 248]
Coordinates of pixel: X:  287 Y:  276
Left Click - HSV : [  0   0 248]
Coordinates of pixel: X:  289 Y:  269
Left Click - HSV : [  0   0 249]
Coordinates of pixel: X:  288 Y:  266
Left Click - HSV : [  0   0 247]
Coordinates of pixel: X:  287 Y:  267
Left Click - HSV : [  0   0 248]
Coordinates of pixel: X:  286 Y:  280


In [ ]:
def binary_mask_to_xy(binary_mask):
    # Find the coordinates of white pixels (value 1)
    white_points = np.argwhere(binary_mask == 255)

    # Create an array to hold the xy coordinates
    xy_array = np.zeros((white_points.shape[0], 2), dtype=int)

    # Fill the array with x, y coordinates
    xy_array[:, 0] = white_points[:, 1]  # x coordinate (column)
    xy_array[:, 1] = white_points[:, 0]  # y coordinate (row)

    return xy_array

def imfill(img):
    # Create a mask larger than the image
    h, w = img.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # Floodfill from the corner
    filled_image = img.copy()
    cv2.floodFill(filled_image, mask, (0,0), 255)

    # Invert the filled image
    filled_image_inv = cv2.bitwise_not(filled_image)

    # Combine the filled image with the original image
    result = img | filled_image_inv

    return result

mypath = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
ihcc_files = [f for f in onlyfiles if f.endswith('IHCC.tif')]
for read_path in ihcc_files:
    final_path = join(mypath, read_path)
    print(f"Processing {final_path}")  # Debugging line to print the file path being processed

    img = cv2.imread(final_path, cv2.IMREAD_COLOR)

    if img is None:
        print(f"Failed to load image at {final_path}")
        continue
    blurred_img = cv2.GaussianBlur(img, (3, 3), 0)
    hsv = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2HSV)

    # Define the range of blue color in HSV
    #low_blue = np.array([55, 50, 50])
    #high_blue = np.array([130, 255, 255])
    low_blue = np.asarray(min_values_list[2])
    high_blue = np.asarray(max_values_list[2])


    # Define the range of red color in HSV
    #low_red = np.array([130, 50, 50])
    #high_red = np.array([180, 255, 255])
    low_red = np.asarray(min_values_list[1])
    high_red = np.asarray(max_values_list[1])

    # Define HSV range for brown color
    #low_brown = np.array([0, 50, 40])
    #high_brown = np.array([22, 100, 240])
    low_brown = np.asarray(min_values_list[0])
    high_brown = np.asarray(max_values_list[0])

    # Create masks based on the color ranges
    blue_mask = cv2.inRange(hsv, low_blue, high_blue)
    filled_blue_mask = imfill(blue_mask)
    red_mask = cv2.inRange(hsv, low_red, high_red)
    filled_red_mask = imfill(red_mask)
    brown_mask = cv2.inRange(hsv, low_brown, high_brown)
    filled_brown_mask = imfill(brown_mask)

    # Create a blank image with the same dimensions as the original image
    overlay = np.zeros_like(img)

    # Assign colors to the overlay based on masks
    overlay[filled_blue_mask != 0] = [255, 0, 0]  # Blue mask as red (BGR format)
    overlay[filled_red_mask != 0] = [0, 0, 255]  # Red mask as blue (BGR format)
    overlay[filled_brown_mask != 0] = [0, 255, 0]  # Brown mask as custom color (BGR format)



    # Combine the overlay with the original image
    masked_image = cv2.addWeighted(img, 0.7, overlay, 0.3, 0)
    out_path = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt1'

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    full_path = join(out_path, read_path)
    cv2.imwrite(full_path, masked_image)
    print(f"Saved processed image to {full_path}")  # Debugging line to confirm save"""